In [ ]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 14.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight') # For plots

In [ ]:
bajaj_df = pd.read_csv('bajaj.csv')
bajaj_df

,Date,Price,Open,High,Low,Vol.,Change %
0,01/29/2024,"1,637.00","1,631.05","1,640.80","1,618.55",940.17K,0.42%
1,01/25/2024,"1,630.15","1,621.75","1,635.00","1,613.70",1.74M,0.57%
2,01/24/2024,"1,620.85","1,594.70","1,622.85","1,578.35",1.39M,2.42%
3,01/23/2024,"1,582.50","1,597.45","1,615.25","1,575.00",3.03M,0.09%
4,01/20/2024,"1,581.00","1,600.50","1,610.95","1,577.75",404.13K,-1.22%
...,...,...,...,...,...,...,...
3877,06/02/2008,67.49,63.86,68.67,61.11,22.88M,6.67%
3878,05/30/2008,63.27,63.11,65.63,57.80,31.12M,1.83%
3879,05/29/2008,62.13,64.51,69.07,59.73,46.03M,-1.33%
3880,05/28/2008,62.97,55.41,65.39,55.41,46.71M,15.56%


In [ ]:
bajaj_df = bajaj_df.drop_duplicates('Date', keep='last')
bajaj_df

,Date,Price,Open,High,Low,Vol.,Change %
0,01/29/2024,"1,637.00","1,631.05","1,640.80","1,618.55",940.17K,0.42%
1,01/25/2024,"1,630.15","1,621.75","1,635.00","1,613.70",1.74M,0.57%
2,01/24/2024,"1,620.85","1,594.70","1,622.85","1,578.35",1.39M,2.42%
3,01/23/2024,"1,582.50","1,597.45","1,615.25","1,575.00",3.03M,0.09%
4,01/20/2024,"1,581.00","1,600.50","1,610.95","1,577.75",404.13K,-1.22%
...,...,...,...,...,...,...,...
3877,06/02/2008,67.49,63.86,68.67,61.11,22.88M,6.67%
3878,05/30/2008,63.27,63.11,65.63,57.80,31.12M,1.83%
3879,05/29/2008,62.13,64.51,69.07,59.73,46.03M,-1.33%
3880,05/28/2008,62.97,55.41,65.39,55.41,46.71M,15.56%


In [ ]:
bajaj_df = bajaj_df.set_index('Date')
bajaj_df.index = pd.to_datetime(bajaj_df.index)

In [ ]:
bajaj_df.sort_index(inplace=True)

In [ ]:
bajaj_df['Price'] = (bajaj_df['Price'].str.split()).apply(lambda x: float(x[0].replace(',', '')))
bajaj_df['Open'] = (bajaj_df['Open'].str.split()).apply(lambda x: float(x[0].replace(',', '')))
bajaj_df['High'] = (bajaj_df['High'].str.split()).apply(lambda x: float(x[0].replace(',', '')))
bajaj_df['Low'] = (bajaj_df['Low'].str.split()).apply(lambda x: float(x[0].replace(',', '')))
bajaj_df['Change %'] = (bajaj_df['Change %'].str.split()).apply(lambda x: float(x[0].replace('%', '')))
bajaj_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3882 entries, 2008-05-27 to 2024-01-29
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Price     3882 non-null   float64
 1   Open      3882 non-null   float64
 2   High      3882 non-null   float64
 3   Low       3882 non-null   float64
 4   Vol.      3882 non-null   object 
 5   Change %  3882 non-null   float64
dtypes: float64(5), object(1)
memory usage: 212.3+ KB


In [ ]:
df = bajaj_df[['Price', 'Open', 'High', 'Low']].copy()
df

,Price,Open,High,Low
Date,,,,
2008-05-27,54.49,49.61,60.02,48.25
2008-05-28,62.97,55.41,65.39,55.41
2008-05-29,62.13,64.51,69.07,59.73
2008-05-30,63.27,63.11,65.63,57.80
2008-06-02,67.49,63.86,68.67,61.11
...,...,...,...,...
2024-01-20,1581.00,1600.50,1610.95,1577.75
2024-01-23,1582.50,1597.45,1615.25,1575.00
2024-01-24,1620.85,1594.70,1622.85,1578.35


In [ ]:
df = df.dropna()
df

,Price,Open,High,Low
Date,,,,
2008-05-27,54.49,49.61,60.02,48.25
2008-05-28,62.97,55.41,65.39,55.41
2008-05-29,62.13,64.51,69.07,59.73
2008-05-30,63.27,63.11,65.63,57.80
2008-06-02,67.49,63.86,68.67,61.11
...,...,...,...,...
2024-01-20,1581.00,1600.50,1610.95,1577.75
2024-01-23,1582.50,1597.45,1615.25,1575.00
2024-01-24,1620.85,1594.70,1622.85,1578.35


In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=c2a616cb9e5531664286fc98ec9f44a792f782a78090e2bb8e979931f8bd78f0
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
# python -m pip install yahoo-finance; pip install yahoo-finance; pip install yfinance --upgrade --no-cache-dir
# import yfinance as yf
# pip install --upgrade ta; pip install ta
import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [ ]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database['Price'][i+dif])-np.log(Database['Price'][i]))
    return np.append(np.repeat(np.nan, dif),Out), Database.index

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))

#Database is calculated
def DatabaseGeneration (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index)
    return Data.dropna()

#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off')
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11

#Fitting of GJR_GARCH(1,1)
def GJR_GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GJR_GARCH11 = arch_model(AR_Data, p=1, o=1, q=1, dist ='t')
    res_GJR_GARCH11 = GJR_GARCH11.fit(disp='off')
    CV_GJR_GARCH11 = res_GJR_GARCH11.conditional_volatility
    For_CV_GJR_GARCH11 = np.array(res_GJR_GARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return GJR_GARCH11, res_GJR_GARCH11, CV_GJR_GARCH11, For_CV_GJR_GARCH11

#Fitting of TARCH(1,1)
def TARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    TARCH11 = arch_model(AR_Data, p=1, o=1, q=1, power=1.0, dist ='t')
    res_TARCH11 = TARCH11.fit(disp='off')
    CV_TARCH11 = res_TARCH11.conditional_volatility
    For_CV_TARCH11 = np.array(res_TARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return TARCH11, res_TARCH11, CV_TARCH11, For_CV_TARCH11

#Fitting of EGARCH(1,1)
def EGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    EGARCH11 = arch_model(AR_Data, dist ='t', vol="EGARCH")
    res_EGARCH11 = EGARCH11.fit(disp='off')
    CV_EGARCH11 = res_EGARCH11.conditional_volatility
    For_CV_EGARCH11 = np.array(res_EGARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return EGARCH11, res_EGARCH11,CV_EGARCH11, For_CV_EGARCH11

#Fitting of Absolute Value GARCH(1,1)
def AVGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    AVGARCH11 = arch_model(AR_Data, dist ='t', power=1)
    res_AVGARCH11 = AVGARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_AVGARCH11 = res_AVGARCH11.conditional_volatility
    For_CV_AVGARCH11 = np.array(res_AVGARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return AVGARCH11, res_AVGARCH11, CV_AVGARCH11, For_CV_AVGARCH11

#Fitting of FIGARCH11(1,1)
def FIGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res_FIGARCH11 = FIGARCH11.fit(disp='off')
    CV_FIGARCH11 = res_FIGARCH11.conditional_volatility
    For_CV_FIGARCH11 = np.array(res_FIGARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11

#AR models are fitted. As requested by arma package, returns are multiplied by 100 in order to improve the fitting process.
#GARCH(1,1), GJR_GARCH(1,1), TARCH(1,1), EGARCH(1,1), AVGARCH(1,1) and FIGARCH(1,1) volatility models are fitted.
#T student is assumed as distribution.
def AR_Models (Data):
    GARCH, GARCH_Parameters, CV_GARCH, For_CV_GARCH = GARCH_Model_Student(Data)
    GJR_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH = GJR_GARCH_Model_Student(Data)
    TARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH = TARCH_Model_Student(Data)
    EGARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH = EGARCH_Model_Student(Data)
    AVGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH = AVGARCH_Model_Student(Data)
    FIGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH  = FIGARCH_Model_Student(Data)
    return GARCH_Parameters, CV_GARCH, For_CV_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH

#MultiHeadSelfAttention
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(query, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(key, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(value, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(concat_attention)  # (batch_size, seq_len, embed_dim)
        return output

#Transformer Keras Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


def Transformer_Model (Shape1, Shape2, HeadsAttention,Dropout, LearningRate):
    #Model struture is defined
    Input = tf.keras.Input(shape=(Shape1,Shape2), name="Input")
    #LSTM is applied on top of the transformer
    X = tf.keras.layers.LSTM(units=16, dropout=Dropout, return_sequences=True)(Input)
    #Tranformer architecture is implemented
    transformer_block_1 = TransformerBlock(embed_dim=16, num_heads=HeadsAttention, ff_dim=8, rate=Dropout)
    X = transformer_block_1(X)
    #Dense layers are used
    X = tf.keras.layers.GlobalAveragePooling1D()(X)
    X = tf.keras.layers.Dense(8, activation=tf.nn.sigmoid)(X)
    X = tf.keras.layers.Dropout(Dropout)(X)
    Output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, name="Output")(X)
    model = tf.keras.Model(inputs=Input, outputs=Output)
    #Optimizer is defined
    Opt = tf.keras.optimizers.Adam(learning_rate=LearningRate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
    #Model is compiled
    model.compile(optimizer=Opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

#It generates the database for fitting transformer. No positional encoding is needed as LSTM plays this role in the model structure
def Transformer_Database (Timestep, XData_AR, YData_AR):
    Features = XData_AR.shape[1]; Sample = XData_AR.shape[0]-Timestep+1
    XDataTrainScaledRNN=np.zeros([Sample, Timestep, Features]); YDataTrainRNN=np.zeros([Sample])
    for i in range(Sample):
        XDataTrainScaledRNN[i,:,:] = XData_AR[i:(Timestep+i)]
        YDataTrainRNN[i] = YData_AR[Timestep+i-1]
    return XDataTrainScaledRNN, YDataTrainRNN

#Database is calculated
def DatabaseGenerationForecast (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index)
    return Data

#Final AR database for forcasting is generated
def DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH):
    Data_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).iloc[(-LagSD+1)]
    Index_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).index[(-LagSD+1)]
    XDataForecast={'SD': Data_Forecast['SD'], 'DailyReturnsOld': Data_Forecast['DailyReturnsOld'],
               'CV_GARCH' : For_CV_GARCH/100, 'CV_GJR_GARCH' : For_CV_GJR_GARCH/100, 'CV_TARCH' : For_CV_TARCH/100,
               'CV_EGARCH' : For_CV_EGARCH/100, 'CV_AVGARCH' : For_CV_AVGARCH/100, 'CV_FIGARCH' : For_CV_FIGARCH/100}
    return pd.DataFrame([XDataForecast], index=[Index_Forecast]), Data_Forecast['DailyReturns']

#Transformed ANN-ARCH model forecast
def T_ANN_ARCH_Forecast (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH,Scaled_Norm, XData_AR, model):
    XDataForecast, ReturnForecast = DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH)
    XDataForecast = pd.concat([XData_AR,XDataForecast])
    XDataForecastTotalScaled = Scaled_Norm.transform(XDataForecast)
    XDataForecastTotalScaled_T, Y_T = Transformer_Database(Timestep, XDataForecastTotalScaled, np.zeros(XDataForecastTotalScaled.shape[0]))
    TransformerPrediction = model.predict(XDataForecastTotalScaled_T)
    return TransformerPrediction[-1][0], XDataForecast.index[-1], TransformerPrediction[0:(XDataForecastTotalScaled_T.shape[0]-1)], ReturnForecast

#It calculates VaR taking into consideration the forecasted sigma to calculate the scale parameter
def T_ANN_ARCH_VaR (Alpha, HistoricalReturns, ForecastedSigma, DF):
    HistoricalMean = np.mean(HistoricalReturns)
    ScaleParameter = np.sqrt((ForecastedSigma**2)*((DF-2)/DF))
    VaR = -t.ppf(Alpha, DF, loc=HistoricalMean, scale=ScaleParameter)
    return VaR

#Formula to calculate the VaR of ARCH models
def VaR_AR_Model (AR_Model,AR_Model_Results,Alpha):
    Cond_Var=AR_Model_Results.forecast(horizon=1).variance.dropna()
    Cond_Mean=AR_Model_Results.forecast(horizon=1).mean.dropna()
    Quantile_Dist=AR_Model.distribution.ppf([Alpha], AR_Model_Results.params[-1:])
    VaR=(-Cond_Mean-np.sqrt(Cond_Var)*Quantile_Dist)/100
    return VaR.values

#Formula to calculate the VaR of all the ARCH models
def VaR_AR_Total(Alpha, GARCH_fit, GJR_GARCH_fit, TARCH_fit, EGARCH_fit, AVGARCH_fit, FIGARCH_fit,GARCH, GJR_GARCH, TARCH, EGARCH, AVGARCH, FIGARCH):
    VaR_GARCH = VaR_AR_Model (GARCH,GARCH_fit,Alpha)
    VaR_GJR_GARCH = VaR_AR_Model (GJR_GARCH,GJR_GARCH_fit,Alpha)
    VaR_TARCH = VaR_AR_Model (TARCH,TARCH_fit,Alpha)
    VaR_EGARCH = VaR_AR_Model (EGARCH,EGARCH_fit,Alpha)
    VaR_AVGARCH = VaR_AR_Model (AVGARCH,AVGARCH_fit,Alpha)
    VaR_FIGARCH = VaR_AR_Model (FIGARCH,FIGARCH_fit,Alpha)
    return {'VaR_GARCH':VaR_GARCH, 'VaR_GJR_GARCH':VaR_GJR_GARCH, 'VaR_TARCH':VaR_TARCH, 'VaR_EGARCH':VaR_EGARCH, 'VaR_AVGARCH':VaR_AVGARCH, 'VaR_FIGARCH':VaR_FIGARCH}

# #Traditional emmbeding position for NLP in transformers
# def position_encoding_init(n_position, d_pos_vec):
#     # keep dim 0 for padding token position encoding zero vector
#     position_enc = np.array([
#         [pos / np.power(10000, 2*i/d_pos_vec) for i in range(d_pos_vec)]
#         if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])
#     position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
#     position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
#     return position_enc
# #Encoding only for the temporal component of the variables.For non-NLP problems.
# def position_encoding_init(n_position, d_pos_vec):
#     position_enc = np.array([
#         [math.pi*(pos/(n_position-1)) for i in range(d_pos_vec)]
#         if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])
#     return np.cos(position_enc)

#Fitting of Transformed ANN-ARCH model and forecasting of the next volatility value
def T_ANN_ARCH_Fit (Data, Lag=1, LagSD=5, Timestep=10, Dropout=0.05, LearningRate=0.01, Epochs=10000, Alpha=0.005, DF=4, BatchSize=64):
    #AR Models are fitted
    GARCH, GARCH_Parameters, CV_GARCH, For_CV_GARCH = GARCH_Model_Student(Data)
    GJR_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH = GJR_GARCH_Model_Student(Data)
    TARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH = TARCH_Model_Student(Data)
    EGARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH = EGARCH_Model_Student(Data)
    AVGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH = AVGARCH_Model_Student(Data)
    FIGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH  = FIGARCH_Model_Student(Data)
    #Database contaning AR models is generated
    Data_AR=pd.concat([Data, CV_GARCH.rename('CV_GARCH')/100, CV_GJR_GARCH.rename('CV_GJR_GARCH')/100, CV_TARCH.rename('CV_TARCH')/100,
                     CV_EGARCH.rename('CV_EGARCH')/100, CV_AVGARCH.rename('CV_AVGARCH')/100, CV_FIGARCH.rename('CV_FIGARCH')/100], axis=1)
    if Data_AR.shape[0]!=Data.shape[0]: print("Error in DB Generation")
    #Original explanatory and response variables are generated
    XData_AR = Data_AR.drop(Data_AR.columns[[0,2]], axis=1);YData_AR = Data_AR['TrueSD']
    #Data is normalized
    Scaled_Norm = preprocessing.StandardScaler().fit(XData_AR); XData_AR_Norm = Scaled_Norm.transform(XData_AR)
    #Data for fitting the transformer model is generated
    XData_AR_Norm_T, YData_AR_Norm_T= Transformer_Database(Timestep, XData_AR_Norm, YData_AR)
    #Model with transformer layer is defined
    model = Transformer_Model(XData_AR_Norm_T.shape[1], XData_AR_Norm_T.shape[2], HeadsAttention=4, Dropout=Dropout, LearningRate=LearningRate)
    model.fit(XData_AR_Norm_T, YData_AR_Norm_T, epochs=Epochs, verbose=0, batch_size=BatchSize); tf.keras.backend.clear_session()
    Forecast, Date_Forecast, TrainPrediction, ReturnForecast = T_ANN_ARCH_Forecast (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH,Scaled_Norm, XData_AR, model)
    VaR = T_ANN_ARCH_VaR(Alpha, Data['DailyReturnsOld'], Forecast,DF)
    return {'T_ANN_ARCH_model':model, 'Forecast_T_ANN_ARCH':Forecast, 'Date_Forecast':Date_Forecast, 'TrainPrediction': TrainPrediction, 'Scaler':Scaled_Norm, 'Forecast_GARCH':For_CV_GARCH, 'Forecast_GJR_GARCH':For_CV_GJR_GARCH, 'Forecast_TARCH':For_CV_TARCH, 'Forecast_EGARCH':For_CV_EGARCH, 'Forecast_AVGARCH':For_CV_AVGARCH, 'Forecast_FIGARCH':For_CV_FIGARCH, 'ReturnForecast':ReturnForecast, 'GARCH_fit': GARCH_Parameters, 'GJR_GARCH_fit':GJR_GARCH_Parameters, 'TARCH_fit':TARCH_Parameters, 'EGARCH_fit':EGARCH_Parameters, 'AVGARCH_fit':AVGARCH_Parameters, 'FIGARCH_fit':FIGARCH_Parameters, 'GARCH': GARCH, 'GJR_GARCH':GJR_GARCH, 'TARCH':TARCH, 'EGARCH':EGARCH, 'AVGARCH':AVGARCH, 'FIGARCH':FIGARCH, 'YData_Train':YData_AR_Norm_T, 'VaR': VaR}
#     return (model, Forecast, Date_Forecast, TrainPrediction, Scaled_Norm, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH, ReturnForecast, GARCH_Parameters, GJR_GARCH_Parameters, TARCH_Parameters, EGARCH_Parameters, AVGARCH_Parameters, FIGARCH_Parameters, GARCH, GJR_GARCH, TARCH, EGARCH, AVGARCH, FIGARCH, YData_AR_Norm_T)

In [ ]:
train_df = df.loc[:'2022-12-30'].reset_index(drop=False)
test_df = df.loc['2023-01-02':].reset_index(drop=False)

In [ ]:
train_df = train_df.set_index('Date')
test_df = test_df.set_index('Date')

In [ ]:
trIndexEndDays = train_df.index
trIndexDates = trIndexEndDays[2000:]

In [ ]:
trIndexDates.shape[0]

1616

In [ ]:
import gc

In [ ]:
Lag=1; LagSD=5; Timestep=10; Dropout=0.15; LearningRate=0.01; Epochs=50; Alpha=0.005; DF=4
DataValidation = DatabaseGeneration(df, Lag, LagSD)
ResultsCollection=pd.DataFrame({'Date_Forecast': [], 'Forecast_T_ANN_ARCH': [],'Forecast_GARCH':[],'Forecast_GJR_GARCH':[], 'Forecast_TARCH':[],'Forecast_EGARCH':[],'Forecast_AVGARCH':[],'Forecast_FIGARCH':[],'ReturnForecast':[],'TrueSD':[], 'VaR_T_ANN_ARCH':[], 'VaR_GARCH':[], 'VaR_GJR_GARCH':[], 'VaR_TARCH':[], 'VaR_EGARCH':[], 'VaR_AVGARCH':[], 'VaR_FIGARCH':[]})


In [ ]:
for i in tqdm(range(trIndexDates.shape[0])):
    #Database is downloaded from yahoo finance and lag of returns defined
    Database=df.loc[(pd.to_datetime(df.index).date >= trIndexDates[i].date()-timedelta(days=650)) & (pd.to_datetime(df.index).date <= trIndexDates[i].date())]
    #Database for fitting the models is generated
    Data = DatabaseGeneration (Database, Lag, LagSD)
    #Fitting of Transformed ANN-ARCH model, ARCH models and forecasting of the next volatility value
    T_ANN_ARCH_Model = T_ANN_ARCH_Fit (Data, Lag, LagSD, Timestep, Dropout, LearningRate, Epochs, Alpha, DF)
    #VaR of ARCH models is computed
    VaR_ARCH_Models=VaR_AR_Total(Alpha, T_ANN_ARCH_Model['GARCH_fit'], T_ANN_ARCH_Model['GJR_GARCH_fit'], T_ANN_ARCH_Model['TARCH_fit'], T_ANN_ARCH_Model['EGARCH_fit'], T_ANN_ARCH_Model['AVGARCH_fit'], T_ANN_ARCH_Model['FIGARCH_fit'],T_ANN_ARCH_Model['GARCH'], T_ANN_ARCH_Model['GJR_GARCH'], T_ANN_ARCH_Model['TARCH'], T_ANN_ARCH_Model['EGARCH'], T_ANN_ARCH_Model['AVGARCH'], T_ANN_ARCH_Model['FIGARCH'])
    #Results are collected
    gc.collect()
    IterResults={'Date_Forecast': T_ANN_ARCH_Model['Date_Forecast'].date(), 'Forecast_T_ANN_ARCH': T_ANN_ARCH_Model['Forecast_T_ANN_ARCH'],'Forecast_GARCH':T_ANN_ARCH_Model['Forecast_GARCH']/100,'Forecast_GJR_GARCH':T_ANN_ARCH_Model['Forecast_GJR_GARCH']/100, 'Forecast_TARCH':T_ANN_ARCH_Model['Forecast_TARCH']/100,'Forecast_EGARCH':T_ANN_ARCH_Model['Forecast_EGARCH']/100,'Forecast_AVGARCH':T_ANN_ARCH_Model['Forecast_AVGARCH']/100,'Forecast_FIGARCH':T_ANN_ARCH_Model['Forecast_FIGARCH']/100,'ReturnForecast':T_ANN_ARCH_Model['ReturnForecast'],'TrueSD':DataValidation[DataValidation.index==pd.to_datetime(T_ANN_ARCH_Model['Date_Forecast'].date())]['TrueSD'][0], 'VaR_T_ANN_ARCH': T_ANN_ARCH_Model['VaR'], 'VaR_GARCH':VaR_ARCH_Models['VaR_GARCH'][0][0], 'VaR_GJR_GARCH':VaR_ARCH_Models['VaR_GJR_GARCH'][0][0], 'VaR_TARCH':VaR_ARCH_Models['VaR_TARCH'][0][0], 'VaR_EGARCH':VaR_ARCH_Models['VaR_EGARCH'][0][0], 'VaR_AVGARCH':VaR_ARCH_Models['VaR_AVGARCH'][0][0], 'VaR_FIGARCH':VaR_ARCH_Models['VaR_FIGARCH'][0][0]}
    ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
    #Results are saved
    ResultsCollection.to_csv('6_TL_GARCH.csv',index=False)

  0%|          | 0/1616 [00:00<?, ?it/s]

14/14 [==============================] - 1s 4ms/step


<ipython-input-19-2e96d6784860>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
  0%|          | 1/1616 [00:13<6:00:11, 13.38s/it]

14/14 [==============================] - 1s 5ms/step


<ipython-input-19-2e96d6784860>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
  0%|          | 2/1616 [00:25<5:32:42, 12.37s/it]

...

<ipython-input-19-2e96d6784860>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
100%|█████████▉| 1614/1616 [15:18:43<01:36, 48.19s/it]

14/14 [==============================] - 1s 5ms/step


<ipython-input-19-2e96d6784860>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
100%|█████████▉| 1615/1616 [15:19:32<00:48, 48.41s/it]

14/14 [==============================] - 1s 6ms/step


<ipython-input-19-2e96d6784860>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
100%|██████████| 1616/1616 [15:20:22<00:00, 34.17s/it]
